In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests, lxml
from lxml import html
import json
from decimal import *
import math
import datetime

In [2]:
ENV = 'prod'
ENV_FILENAME = 'prod.env' if ENV == 'prod' else 'dev.env'
AIRTABLE_SHEET_NAME = 'Company Data ENTRY TABLE' if ENV == 'prod' else 'dev db'

In [3]:
env_vars = {}

with open(ENV_FILENAME) as f:
    for line in f:
        if line.startswith('#') or not line.strip():
            continue
        key, value = line.strip().split('=')
        env_vars[key] = value

AIRTABLE_TOKEN = env_vars["AIRTABLE_TOKEN"]
AIRTABLE_BASE_ID = env_vars["AIRTABLE_BASE_ID"]
AIRTABLE_URL = f"https://api.airtable.com/v0/{AIRTABLE_BASE_ID}"

In [4]:
class statistics:
    yfinance_url = "https://finance.yahoo.com/"
    def __init__(self, symbol):
        '''

        :param symbol: stock symbol in all caps
        please note that any Canadian TSX stocks are followed with ".TO"     
        check the relevant URL for formatting.
        '''
        self.symbol = symbol.upper()
        self.path = "quote/{0}/key-statistics?p={0}".format(symbol)
        self.url = self.yfinance_url + self.path
        self.methods = ['scrape_page', 'label_stats']
        self.attributes = ['self.symbol', 'self.path', 'self.url',
                          'self.methods', 'self.hdrs','self.valuation']
#                           'self.fiscal_year', 'self.profitability',
#                           'self.manager_effect','self.income_statement',
#                           'self.balance_sheet', 'self.cash_statement',
#                           'self.price_history', 'self.share_stats',  
#                           'self.dividendSplit']
        self.hdrs = {"authority": "finance.yahoo.com",
                     "method": "GET",
                     "path": self.path,
                     "scheme": "https",
                     "accept": "text/html,application/xml;q=0.9",
                     "accept-encoding": "gzip, deflate, br",
                     "accept-language": "en-US,en;q=0.9",
                     "referer": self.yfinance_url,
                     "sec-fetch-mode": "navigate",
                     "sec-fetch-site": "same-origin",
                     "sec-fetch-user": "?1",
                     "upgrade-insecure-requests": "1",
                     "user-agent": "Mozilla/5.0 (Windows NT 10.0;)"}
    def scrape_page(self):
        '''

        :return: scrapes the content of the class URL,
                   using headers defined in the init function,
                   returning a byte string of html code.
        '''
        page = requests.get(self.url, headers=self.hdrs)
        soup = BeautifulSoup(page.content, 'lxml')
        tables = soup.find_all('table')
        iterator = range(0, len(tables))
        function = lambda x: pd.read_html(str(tables[x]))
        table_list = list(map(function, iterator))[0]
#         print('table: ', table_list[0])
        market_cap = table_list[0].iloc[0][1]
        return market_cap
    def label_stats(self, table_list):
        '''
        :param table_list: uses the output of the scrape_page method
        :return: creates attributes for the statistics class object,
                 uses indexLabel method to label columns and set the dataframes' index
        
        '''
        iterator = [table_list[i][0] for i in range(0, len(table_list))]
        
        table_list = list(map(lambda df: self.__indexLabel__(df), iterator))
#         self.valuation, self.fiscal_year, self.profitability, self.manager_effect, \
#         self.income_statement, self.balance_sheet, self.cash_statement, \
#         self.price_history, self.share_stats, self.dividendSplit = table_list
        
        self.valuation = table_list
        print(self.valuation)
        return table_list
    def __indexLabel__(self, df):
        '''
        
        :param df: Takes a dataframe as input.
        :return: returns a dataframe with column labels and a set index.
        
        '''
        df.columns = ['Measure', 'Value']
        df = df.set_index('Measure')
        return df

In [5]:
class RealTimeCurrencyConverter():
    def __init__(self, url):
        self.data = requests.get(url).json()
        self.rates = self.data['rates']
        self.date = self.data['date']
    def convert(self, amount, currency):
        if currency == 'USD':
            return amount
        conversion_rate = self.rates[currency]
        usd_amount = amount / self.rates[currency]
        return usd_amount

In [6]:
exchange_rate_url = 'https://api.exchangerate-api.com/v4/latest/USD'
converter = RealTimeCurrencyConverter(exchange_rate_url)

In [7]:
unit_multiplier = {
    'K': 1000,
    'M': 1000000,
    'B': 1000000000,
    'T': 1000000000000
}
def convert_mc_actual_number(mc_number, mc_number_unit):
    return mc_number * unit_multiplier[mc_number_unit]

In [8]:
def convert_formatted_mc_actual_number(usd_amount):
    oneplace = Decimal(10) ** -1
    if usd_amount >= unit_multiplier['T']:
        formatted_usd_amount = usd_amount/unit_multiplier['T']
        formatted_usd_amount_one_decimal = Decimal(formatted_usd_amount).quantize(oneplace, rounding=ROUND_UP)
        return str(formatted_usd_amount_one_decimal) + ' T'
    elif usd_amount >= unit_multiplier['B']:
        formatted_usd_amount = usd_amount/unit_multiplier['B']
        formatted_usd_amount_one_decimal = Decimal(formatted_usd_amount).quantize(oneplace, rounding=ROUND_UP)
        return str(formatted_usd_amount_one_decimal) + ' B'
    elif usd_amount >= unit_multiplier['M']:
        formatted_usd_amount = usd_amount/unit_multiplier['M']
        formatted_usd_amount_one_decimal = Decimal(formatted_usd_amount).quantize(oneplace, rounding=ROUND_UP)
        return str(formatted_usd_amount_one_decimal) + ' M'
    else:
        formatted_usd_amount = usd_amount/unit_multiplier['K']
        formatted_usd_amount_one_decimal = Decimal(formatted_usd_amount).quantize(oneplace, rounding=ROUND_UP)
        return str(formatted_usd_amount_one_decimal) + ' K'

In [9]:
def get_currency(stock_symbol):
    if 'HK' in stock_symbol:
        return 'HKD'
    elif 'SS' in stock_symbol or 'SZ' in stock_symbol:
        return 'CNY'
    elif 'TW' in stock_symbol:
        return 'TWD'
    else:
        return 'USD'

In [10]:
def get_note(currency, time):
    if currency == 'USD':
        return 'Valuation is reported as market capitalization. It was last updated on {}.'.format(time)
    elif currency == 'HKD':
        return 'Valuation is reported as market capitalization. It was converted from {} using the exchange rate from {}.'.format(currency, time)
    elif currency == 'CNY':
        return 'Valuation is reported as market capitalization. It was converted from RMB using the exchange rate from {}.'.format(time)
    elif currency == 'TWD':
        return 'Valuation is reported as market capitalization. It was converted from TWD using the exchange rate from {}.'.format(time)

In [11]:
def prioritize(ticker):
    tickers_for_company = ticker.split(',')

    if 'NYSE' in ticker:
        nyse_ticker = list(filter(lambda t: ('NYSE' in t), tickers_for_company))[0]
        return nyse_ticker.split(':')[1].strip()
    elif 'NASDAQ' in ticker:
        nasdaq_ticker = list(filter(lambda t: ('NASDAQ' in t), tickers_for_company))[0]
        return nasdaq_ticker.split(':')[1].strip()
    elif 'HK' in ticker:
        return list(filter(lambda t: ('HK' in t), tickers_for_company))[0].strip()
    elif 'SS' in ticker:
        return list(filter(lambda t: ('SS' in t), tickers_for_company))[0].strip()
    elif 'SZ' in ticker:
        return list(filter(lambda t: ('SZ' in t), tickers_for_company))[0].strip()
    elif 'TW' in ticker:
        return list(filter(lambda t: ('TW' in t), tickers_for_company))[0].strip()

In [12]:
def pull_from_yfinance_and_save_as_json(records):
    filtered_records = list(filter(lambda record: '_ticker_symbol TEST' in record['fields'], records))

    patch_records = []
    for record in filtered_records:
        now = datetime.datetime.now()
        time = now.strftime("%B %d, %Y %H:%M:%S")
        del record['createdTime']
        
        unparsed_ticker = record['fields']['_ticker_symbol TEST']
        stock_symbol = prioritize(unparsed_ticker)
        
        try:
            market_cap = statistics(stock_symbol).scrape_page()
            print(stock_symbol, ': ', market_cap)
        except:
            pass

        if isinstance(market_cap, str) and market_cap[-1] in ['K', 'M', 'B', 'T']:
            mc_number = float(market_cap[:-1])
            mc_number_unit = market_cap[-1] # M for million, B for billion, T for trillion
            mc_actual_number = convert_mc_actual_number(mc_number, mc_number_unit)
            currency = get_currency(stock_symbol)
            usd_amount = converter.convert(mc_actual_number, currency)
            
            formatted_mc = 'USD ' + convert_formatted_mc_actual_number(usd_amount)
            record['fields']['company_valuation'] = formatted_mc
            note = get_note(currency, time)
            record['fields']['company_valuation_notes'] = note
            patch_record = {
                "id": record['id'],
                "fields": {
                    "company_valuation": formatted_mc,
                    "company_valuation_notes": note
                }
            }
            patch_records.append(patch_record)
    
    return patch_records

In [13]:
def save_in_json(input, filename):        
    with open(filename, 'w', encoding='utf-8') as jsonfile:
        jsonfile.write(json.dumps(input, indent=4))

In [14]:
def get_airtable_data(sheet_name, offset=None):
    url = f"{AIRTABLE_URL}/{sheet_name}"
    headers = {
        'Authorization': f'Bearer {AIRTABLE_TOKEN}',
        'Content-Type': 'application/json'
    }
    params = {}
    if offset:
        params["offset"] = offset
    
    response = requests.request("GET", url, headers=headers, params=params)
    return response

In [15]:
def update_airtable_data(sheet_name, version):
    url = f"{AIRTABLE_URL}/{sheet_name}"
    headers = {
        'Authorization': f'Bearer {AIRTABLE_TOKEN}',
        'Content-Type': 'application/json'
    }
    
    f = open ('patch_{}.json'.format(version), "r")
    payload = json.loads(f.read())
    response = requests.request("PATCH", url, headers=headers, data=json.dumps(payload))

In [16]:
sheet_name = AIRTABLE_SHEET_NAME
airtable_response = get_airtable_data(sheet_name).json()
all_records = []
records = get_airtable_data(sheet_name)
all_records.extend(records.json()['records'])

while "offset" in records.json():
    records = get_airtable_data(sheet_name, records.json()["offset"])
    all_records.extend(records.json()['records'])
    
save_in_json(all_records, 'get.json')
updated_records = pull_from_yfinance_and_save_as_json(all_records)

for count, record in enumerate(updated_records, 1):
    if count % 10 == 0:
        json_records = updated_records[count - 10:count]
        updated_requests = {'records': json_records}
        version = int(count/10)
        save_in_json(updated_requests, 'patch_{}.json'.format(version))
        update_airtable_data(sheet_name, version)

version = int(count/10 + 1)
count = count % 10
json_records = updated_records[-count:]
updated_requests = {'records': json_records}
save_in_json(updated_requests, 'patch_{}.json'.format(version))
update_airtable_data(sheet_name, version)

print("Updated {} of records.".format(len(updated_records)))

000625.SZ :  90.38B
0390.HK :  189.33B
1024.HK :  302.63B
002074.SZ :  57.45B
KRKR :  31.61M
002714.SZ :  259.45B
IQ :  3.24B
0031.HK :  1.48B
000333.SZ :  398.13B
MF :  55.66M
TCOM :  13.96B
0763.HK :  122.76B
JD :  79.51B
0175.HK :  137.09B
1186.HK :  123.41B
300014.SZ :  152.13B
1062.HK :  255.39M
3333.HK :  21.79B
JKS :  2.79B
BABA :  241.21B
2318.HK :  894.18B
1772.HK :  192.27B
TSM :  463.82B
603501.SS :  143.52B
601012.SS :  415.07B
MFH :  17.43M
2150.HK :  8.30B
300274.SZ :  126.53B
300450.SZ :  77.63B
600392.SS :  31.50B
ZNH :  13.23B
SHI :  4.18B
PDD :  48.79B
CMCM :  101.02M
FEDU :  20.23M
1398.HK :  1.84T
2103.HK :  1.79B
600519.SS :  2.26T
SHI :  4.18B
2618.HK :  89.17B
3328.HK :  409.31B
SVA :  460.25M
EH :  466.22M
002340.SZ :  36.50B
CEA :  11.14B
0525.HK :  15.40B
9888.HK :  335.32B
601888.SS :  343.15B
002156.SZ :  18.77B
RYB :  LyondellBasell Industries NV
UMC :  21.44B
TME :  6.90B
000100.SZ :  62.58B
0941.HK :  1.12T
6770.TW :  211.25B
002024.SZ :  23.37B
PTR :  13